In [3]:
import numpy as np 
import pandas as pd 
import urllib.request
from urllib.parse import quote
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

In [4]:
base_url = 'https://dogpre.com'
page_url = '/category/036004?page=0'
url = base_url + page_url
with urllib.request.urlopen(url) as response:
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')

In [5]:
tmp = soup.select_one('.jss179')
lis = tmp.find_all('a')
len(lis)

20

In [6]:
prod_url = lis[0].attrs['href']
prod_url

'/product/23333'

In [7]:
url = base_url + prod_url
with urllib.request.urlopen(url) as response:
    html = response.read()
    prd_soup = BeautifulSoup(html, 'html.parser')

In [8]:
# 상품명
info = prd_soup.select_one('.jss142')
word = info.find('h2').get_text()
word

'네츄럴코어 홀리스틱 베네 M32 멀티프로테인 7.2kg'

In [9]:
# 정가, 판매가
price = info.select_one('.jss198')
price

<div class="jss198"><dl class="jss199"><dt class="jss200">정가</dt><dd class="jss201"><s>49,000<!-- -->원</s></dd></dl><dl class="jss199"><dt class="jss200">판매가</dt><dd class="jss201"><span><span>38,000<!-- -->원</span></span></dd></dl></div>

In [10]:
full_price = int(price.find('s').get_text().replace(',', '').replace('원', ''))
selling_price = int(price.find('span').get_text().replace(',', '').replace('원', ''))
full_price, selling_price

(49000, 38000)

In [11]:
# 할인받은 금액, 할인율
discount = info.select_one('.jss205').get_text()
discount

'11,000원 (22%)'

In [12]:
diff_price, disc_rate = discount.split(maxsplit=1)
diff_price = int(diff_price.replace(',', '').replace('원', ''))
disc_rate = disc_rate[1:-1]
diff_price, disc_rate

(11000, '22%')

In [13]:
# 첫 번째 페이지
name_list = []; full_price_list = []; selling_price_list = []
diff_price_list = []; disc_rate_list = []
tmp = soup.select_one('.jss179')
lis = tmp.find_all('a')
base_url = 'https://dogpre.com'

for li in tqdm_notebook(lis):
    #print(i)        # 몇 번째 상품에서 에러가 나는지 확인하는 방법, enumerate와 같이 써야함
    prod_url = li.attrs['href']
    url = base_url + prod_url
    with urllib.request.urlopen(url) as response:
        html = response.read()
        prd_soup = BeautifulSoup(html, 'html.parser')

    # 상품명
    info = prd_soup.select_one('.jss142')
    word = info.find('h2').get_text()
    name_list.append(word)
    # 정가, 판매가
    price = info.select_one('.jss198')
    if price:
        price = info.select_one('.jss198')
    else:
        price = info.select_one('.jss252')
    full_price = int(price.find('s').get_text().replace(',', '').replace('원', ''))
    selling_price = int(price.find('span').get_text().replace(',', '').replace('원', ''))
    full_price_list.append(full_price); selling_price_list.append(selling_price)
    # 할인받은 금액, 할인율
    discount = info.select_one('.jss205').get_text()
    if '%' in discount:
        discount = info.select_one('.jss205').get_text()
    else:
        discount = info.select_one('.jss259').get_text()
    diff_price, disc_rate = discount.split(maxsplit=1)
    diff_price = int(diff_price.replace(',', '').replace('원', ''))
    disc_rate = disc_rate[1:-1]
    diff_price_list.append(diff_price); disc_rate_list.append(disc_rate)

In [14]:
product_df = pd.DataFrame({
    '상품명': name_list,
    '원가': full_price_list,
    '판매가': selling_price_list,
    '할인 받은 금액': diff_price_list,
    '할인율': disc_rate_list
})
product_df

,상품명,원가,판매가,할인 받은 금액,할인율
0,네츄럴코어 홀리스틱 베네 M32 멀티프로테인 7.2kg,49000,38000,11000,22%
1,네츄럴코어 유기농 에코 70B 연어 7kg,60000,40000,20000,33%
2,뉴트리나 건강백서 건강한 관절 2kg,20000,15000,5000,25%
3,now 그레인프리 어덜트 스몰브리드 11.3kg,120000,108000,12000,10%
4,로얄캐닌 엑스스몰 어덜트 4.5kg 기획팩,54000,46900,7100,13%
5,now 그레인프리 어덜트 스몰브리드 2.72kg,42000,37800,4200,10%
6,ANF 유기농 6Free 플러스 소고기&연어 5.6kg,45000,39000,6000,13%
7,now 그레인프리 시니어 스몰브리드 2.72kg,42000,37800,4200,10%
8,ANF 유기농 6Free 플러스 오리고기&연어 5.6kg,45000,39000,6000,13%
9,네츄럴코어 유기농 에코 9a 시니어 오리&고구마 6kg,60000,52000,8000,13%


In [19]:
# 모든 페이지
base_url = 'https://dogpre.com'
name_list = []; full_price_list = []; selling_price_list = []
diff_price_list = []; disc_rate_list = []
total_page = 26

In [21]:
for page in range(total_page):
    page_url = f'/category/036004?page={page}'
    url = base_url + page_url
    with urllib.request.urlopen(url) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
    tmp = soup.select_one('.jss179')
    lis = tmp.find_all('a')

    for i, li in enumerate(lis):
        print(i)        # 몇 번째 상품에서 에러가 나는지 확인하는 방법, enumerate와 같이 써야함
        prod_url = li.attrs['href']
        url = base_url + prod_url
        with urllib.request.urlopen(url) as response:
            html = response.read()
            prd_soup = BeautifulSoup(html, 'html.parser')

        # 상품명
        info = prd_soup.select_one('.jss142')
        word = info.find('h2').get_text()
        name_list.append(word)
        # 정가, 판매가
        price = info.select_one('.jss198')
        if price:
            price = info.select_one('.jss198')
            full_price = int(price.find('s').get_text().replace(',', '').replace('원', ''))
            full_price_list.append(full_price)
        else:
            if '정가' in price.get_text():
                price = info.select_one('.jss252')
                full_price = int(price.find('s').get_text().replace(',', '').replace('원', ''))
                full_price_list.append(full_price)
            else:
                price = info.select_one('.jss512')
                full_price_list.append(' ')
        selling_price = int(price.find('span').get_text().replace(',', '').replace('원', ''))
        full_price_list.append(full_price)
        selling_price_list.append(selling_price)
        # 할인받은 금액, 할인율
        discount = info.select_one('.jss205').get_text()
        if '%' in discount:
            discount = info.select_one('.jss205').get_text()
        else:
            discount = info.select_one('.jss259').get_text()
        diff_price, disc_rate = discount.split(maxsplit=1)
        diff_price = int(diff_price.replace(',', '').replace('원', ''))
        disc_rate = disc_rate[1:-1]
        diff_price_list.append(diff_price); disc_rate_list.append(disc_rate)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9


AttributeError: 'NoneType' object has no attribute 'get_text'

In [ ]:
# 데이터프레임에 넣을 리스트들의 길이가 같은지 확인
len(name_list), len(manufacturer_list), len(origin_list), len(full_price_list), len(disc_price_list)

In [ ]:
product_df = pd.DataFrame({
    '상품명': name_list,
    '제조사': manufacturer_list,
    '원산지': origin_list,
    '원가': full_price_list,
    '할인가': disc_price_list
})

In [ ]:
product_df.tail()